# Scooter Data Analysis

## Data loading

In [1]:
from data_transcoders import *

In [2]:
url = r"http://158.160.79.188:8000/GetAll"

frames_by_session_id, sessions_by_session_id = load_frames_by_url(url)

In [3]:
batch_size = 100
batches = list()

for session_id in frames_by_session_id.keys():
    print(session_id)
    batches.extend(unite_frames_to_frame_batches(frames_by_session_id[session_id], sessions_by_session_id[session_id], batch_size))

batches

1702884561711
1702885369203
1702885959961
1702886316591
1702886538311
1702887052896
1702887153353
1702887303208
1702970925611
1702971382346
1702977187278
1702977212729


In [15]:
df = frame_batches_to_dataframe(batches)

In [16]:
df

,batch_size,average_speed,min_speed,max_speed,average_acceleration,min_acceleration,max_acceleration,average_angle_speed,min_angle_speed,max_angle_speed,time_delta,ride_mode
0,100,0.009228,0.000000,0.013119,3.592625,0.195034,13.159453,0.557445,0.029589,2.728580,9900,0
1,100,0.015129,0.008014,0.022040,8.177330,1.215224,47.966851,0.410825,0.029894,1.126646,9900,0
2,100,0.016062,0.007715,0.025092,4.187579,0.600634,16.841888,0.292457,0.048358,0.906830,9899,0
3,100,0.057820,0.025092,0.059306,3.968728,0.435593,11.290800,0.395382,0.048499,1.593057,21189,0
4,100,0.045073,0.000000,0.059306,3.710081,0.920081,12.775723,0.512588,0.040512,1.352837,10103,0
...,...,...,...,...,...,...,...,...,...,...,...,...
105,100,0.000000,0.000000,0.000000,0.219932,0.094004,0.340720,0.038016,0.020834,0.061590,181,0
106,100,0.000000,0.000000,0.000000,0.657517,0.106141,1.562072,0.231614,0.015565,0.659774,216,0
107,44,0.000000,0.000000,0.000000,0.843267,0.386562,1.128196,0.382611,0.057807,0.572352,56,0
108,44,0.000000,0.000000,0.000000,0.020482,0.006014,0.049392,0.001434,0.000498,0.003447,4291,1
